In [21]:
import requests
import json
import pandas as pd
import pickle
import neattext.functions as nfx

embeddings model

In [2]:
#pip install -U sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer, util

sentence_transformer_model = SentenceTransformer("thenlper/gte-large")

c:\Users\haallat\Anaconda3\envs\tf\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\haallat\Anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def get_embeddings(text):
    tokvecs = sentence_transformer_model.encode(text, normalize_embeddings=True)
    return [float(e) for e in tokvecs]

fetch a scanR publication

In [5]:
url="https://scanr.enseignementsup-recherche.gouv.fr/api/scanr-publications/_search?filter=externalIds.type:doi"

In [6]:
response = requests.get(url, verify=False)

c:\Users\haallat\Anaconda3\envs\tf\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scanr.enseignementsup-recherche.gouv.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
response

<Response [200]>

In [8]:
page=response.json()

In [9]:
page.get('hits').get('hits')[3].get('_source').get('title')

{'default': 'Identification of healthy elderly fallers and non-fallers by gait analysis under dual-task conditions',
 'en': 'Identification of healthy elderly fallers and non-fallers by gait analysis under dual-task conditions'}

In [10]:
externalIds=page.get('hits').get('hits')[3].get('_source').get('externalIds')
externalIds

[{'type': 'doi', 'id': '10.1191/0269215506cr929oa'}]

In [11]:
doi=[x for x in externalIds if x['type']=='doi'][0]['id']
doi

'10.1191/0269215506cr929oa'

In [30]:
doi='10.1016/j.jrurstud.2016.09.009'  #ex de publi tagué "verte"

In [31]:
url_OA=f"https://api.openalex.org/works?filter=doi:{doi}"
response_OA = requests.get(url_OA)
data = response_OA.json()

In [32]:
topics=data.get('results')[0].get('topics')
topics_names=' '.join([topic.get('display_name') for topic in topics])
topics_names

'Drivers and Impacts of Tropical Deforestation Agricultural Innovation and Livelihood Diversification Global Analysis of Ecosystem Services and Land Use'

In [33]:
title=data.get('results')[0].get('title')
title

'Sustainable development policies and the spread of land-sharing practices – A statistical assessment in a frontier region of the Brazilian Amazon'

In [34]:
locations=data.get('results')[0].get('locations')
if (locations!=[]):
    for k in range(len(locations)):
        if pd.isna(locations[k].get('source'))==False:
            if locations[k].get('source').get('type')=='journal':
                locations_names=locations[k].get('source').get('display_name')
                locations_id=locations[k].get('source').get('issn_l') 
                break
print(locations_names,locations_id)

Journal of Rural Studies 0743-0167


In [35]:
model = pickle.load(open("ipcc_model_rd_forest.pkl", 'rb'))

c:\Users\haallat\Anaconda3\envs\tf\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\haallat\Anaconda3\envs\tf\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.3.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [36]:
test=pd.DataFrame(data={
    'title': [title],
    'topics': [topics_names],
    'locations_names':[locations_names],
    'locations_ids':[locations_id]
})

In [37]:
test

,title,topics,locations_names,locations_ids
0,Sustainable development policies and the sprea...,Drivers and Impacts of Tropical Deforestation ...,Journal of Rural Studies,0743-0167


vecyorization sur 1024 dimensions

In [38]:
test['title_vect']=test['title'].apply(lambda x: get_embeddings(nfx.remove_stopwords(x)))

In [39]:
test['topics_vect']=test['topics'].apply(lambda x: get_embeddings(nfx.remove_stopwords(x)))

In [40]:
test['locations_names_vect']=test['locations_names'].apply(lambda x: get_embeddings(nfx.remove_stopwords(x.replace('Journal',''))))

In [41]:
test['locations_ids_vect']=test['locations_ids'].apply(lambda x: get_embeddings(nfx.remove_stopwords(x)))

In [42]:
dict_vect={}
for y in ['title','topics','locations_ids','locations_names']:
    dict_vect[y]=test[f"{y}_vect"].apply(pd.Series)
    dict_vect[y].columns=[f"{y}_dim_{x}" for x in dict_vect[y].columns]

In [43]:
features_df = pd.concat(list(dict_vect.values()), axis=1)

In [44]:
features_df

,title_dim_0,title_dim_1,title_dim_2,title_dim_3,title_dim_4,title_dim_5,title_dim_6,title_dim_7,title_dim_8,title_dim_9,...,locations_names_dim_1014,locations_names_dim_1015,locations_names_dim_1016,locations_names_dim_1017,locations_names_dim_1018,locations_names_dim_1019,locations_names_dim_1020,locations_names_dim_1021,locations_names_dim_1022,locations_names_dim_1023
0,-0.013902,0.040763,-0.027236,-0.007924,-0.002478,0.014867,-0.000579,0.039965,0.031032,0.038083,...,0.06285,-0.050759,-0.035414,-0.052723,0.04989,0.073172,0.01171,-0.023596,-0.004724,-0.002684


0 -> pas vert
<br />
1 -> vert

In [45]:
model.predict(features_df)

array([1], dtype=int64)